## Labeling using regex groups

In [3]:
from pathlib import Path
import pandas as pd

# load full_df if not loaded
DATA_PATH = '../competitors-xgb/data'
file_path = (
    Path(DATA_PATH) / 'tables_OZ_geo_5500' /
    'processed' / 'OZ_geo_5500.csv'
)
full_df = pd.read_csv(file_path) \
    .drop(columns='sales.1') # TODO: not include this column when creating the dataframe in make_OZ_geo_5500

# combine 'name' and 'description'
full_df.description = full_df.description.fillna('')
full_df.name.isna().sum(), full_df.description.isna().sum()
full_df['name_and_description'] = full_df['name'] + '.\n' + full_df['description']
pd.set_option('display.max_colwidth', None)
full_df[['sku', 'name_and_description']].head(1)

,sku,name_and_description
0,1871769771,"Карты МИРА и РОССИИ настенные политические,160х102 см, Комплект школьных карт для детей / 2024 г.\nПредставляем вашему вниманию уникальный набор карт, состоящий из Географической карты России и Настенной карты мира. Каждая карта имеет размер 102х160 см и масштаб 1:5 500 000 для России и 1:22 000 000 для мира. Эти современные издания 2024 года отражают актуальные изменения с новыми границами и помогут вам в образовательных целях, декоре и повседневной жизни.\nГеографическая карта России — идеальный помощник для школьников и любителей географии. Она прекрасно иллюстрирует все регионы страны, их особенности и природные зоны. Настенная карта России характеризуется четкой и детализированной графикой, что делает её удобной для изучения. Эта физическая карта позволяет лучше понять разнообразие природных зон России и узнать о местной флоре и фауне. Яркие цвета и четкие обозначения обеспечивают высокую читаемость, что позволяет легко находить нужные области и изучать их.\nНастенная карта мира станет отличным выбором для образовательных целей и домашнего использования. Она помогает изучать страны, их границы и столицы, развивая пространственное мышление. Политическая карта мира выполнена в ярких цветах и с четкими границами, включая моря, океаны и флаги стран. Она подойдет как для школьных кабинетов, так и для стильного оформления интерьера вашего дома.\nОбъединяя два товара, совместите полезное с эстетическим — карты можно использовать как современные учебные пособия для школьников и взрослых, а также как элементы декора. Этот набор станет прекрасным подарком на 1 сентября, Новый год и другие праздники. Ламинированная поверхность защищает карты от повреждений, делая их долговечными и удобными для использования.\nВыбирая набор карт, вы обеспечиваете полезный и функциональный инструмент для изучения нашей страны и мира, что делает их незаменимыми в учебном процессе и повседневной жизни."


In [7]:
# Extract size

import pandas as pd
import numpy as np

# Regex pattern: match width x height with optional unit (мм, см, м)
size_pattern = r'(\d+(?:[.,]\d+)?)\s*[xхХ×\*]\s*(\d+(?:[.,]\d+)?)(?:\s*(мм|см|м))?'

# Extract all size_matches_df from 'name_and_description'
size_matches_df = full_df['name_and_description'].str.extractall(size_pattern)

# Normalize numbers (comma to dot) and convert to float
size_matches_df[0] = size_matches_df[0].str.replace(',', '.', regex=False).astype(float)
size_matches_df[1] = size_matches_df[1].str.replace(',', '.', regex=False).astype(float)

# Standardize unit column
size_matches_df[2] = size_matches_df[2].str.lower().fillna('см')  # default to cm

# Rename columns
size_matches_df.columns = ['width_raw', 'height_raw', 'unit_raw']

# Bigger size goes first -> allow symmetrical sizes (100x60 = 60x100)
size_matches_df[["width_raw","height_raw"]] = np.sort(size_matches_df[["width_raw","height_raw"]].values, axis=1)[:,::-1]

# Convert to centimeters
conversion_factors = {'мм': 0.1, 'см': 1, 'м': 100}
size_matches_df['factor'] = size_matches_df['unit_raw'].map(conversion_factors)
size_matches_df['width'] = size_matches_df['width_raw'] * size_matches_df['factor']
size_matches_df['height'] = size_matches_df['height_raw'] * size_matches_df['factor']
size_matches_df = size_matches_df.astype({'width': int, 'height': int})

# Add SKU info
size_matches_df = size_matches_df.reset_index().merge(full_df[['sku']], left_on='level_0', right_index=True)

# Final output
size_matches_df = size_matches_df[['sku', 'width', 'height', 'unit_raw', 'width_raw', 'height_raw']]

In [ ]:
# View extracted sizes 
print(f'SKU with extracted sizes: {size_matches_df.sku.nunique()}')

nosize_sku = set(full_df['sku'].tolist()) - set(size_matches_df['sku'].tolist())
nosize_sku = list(nosize_sku)
print(f'SKU without extracted sizes: {len(nosize_sku)}')

pd.reset_option('display.max_rows')
display(size_matches_df)

SKU with extracted sizes: 4957
SKU without extracted sizes: 605


,sku,width,height,unit_raw,width_raw,height_raw
0,1871769771,160,102,см,160.0,102.0
1,1871769771,160,102,см,160.0,102.0
2,1679550303,70,50,м,0.7,0.5
3,1679550303,70,50,м,0.7,0.5
4,1679550303,70,50,см,70.0,50.0
...,...,...,...,...,...,...
7914,166584090,122,79,см,122.0,79.0
7915,166451882,60,40,см,60.0,40.0
7916,154409524,83,39,см,83.0,39.3
7917,147896031,80,60,см,80.0,60.0


In [11]:
# Define regex pattern dictionary per each sku

ALL_PATTERNS_DICT = {
    # DONE
    1871769771: {
        True: {
            "мира",
            "россии",
            "настенн",
            "политическ",
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            # r"|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b"
        },
        False: {
            # 'тубус',
            # "отвес|рейк",
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # # DONE (no positives in dataset)
    # 1679550303: {
    #     True: {
    #         "москв",
    #         "метро",
    #     },
    #     False: {
    #         "фальцованая|складная",
    #         "настенн"
    #     }
    # },

    # DONE
    # False Positives: 
    1200553001: {
        True: {
            "политическая",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк',
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    922231521: {
        True: {
            "политическ",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    # False Positives: 
    # - 1438364140: выбор карты 99x160 или 100x60 (101x60)
    922230517: {
        True: {
            "политическ",
            "мира",
            "настенн",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    # False Positives: 
    # - 166584091: описание нескольких карт в ассортименте (в т.ч. скретч-карта)
    922230183: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    922229770: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'детская',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    824158517: {
        True: {
            # "географическая|по географии",
            "политическая",
            'мир',
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|двух?сторонняя карта",

            r"(?:\s+(?!ламинация\b)[\w-]+(?:\s*[,.:;]\s*)?)"  # ← allow an optional , . : ; between the word and target
            r"\s+двух?сторонняя|стороны\b"                            # “двусторонняя” / “двухсторонняя”
            r"(?!\s+ламинация\b)",                            # not immediately followed by “ламинация”

            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'политико-административная|административно-политическая',
            "карта россии",
            'иллюстрированн|с иллюстрациями',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    508612558: {
        True: {
            "москв",
            "настен",
            "метро",
            "с линиями метро"
        },
    },


    # DONE
    508611672: {
        True: {
            'москв',
            'московск.*област',
        },
        False: {
            'округ',
            'Большая Москва', # HACK: too local
        }
    },

    # DONE
    507113963: {
        True: {
            'настенн',
            'европы',
            'политическ',
        },
    },

    # DONE
    # False Negatives:
    # - не указаны размеры, по хактеристикам подходит: 1538773969, 1043438100, 1126764501, 1129697481, 1215475736, 1573135174
    494562010: {
        True: {
            "мира? и|и мира?",
            "росси",
            "складн|фальцован",
            "политическ",
            # 'политико-административная|административно-политическая',
        },
        False: {
            # "отвес|рейк",
            # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя карта|шт[\.|ук|уки]",

            'физическ',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
        }
    },

    # # DONE (no positives in dataset)
    # 492260072: {
    #     True: {
    #         "(?:атлас|карта) автомобильных дорог росси", # False Negative: 1710867998 - указано 'карта автомобильных дорог России' в карте Москвы (на сайте уже нет)
    #     },
    # },

    # DONE
    491279127: {
        True: {
            "политическ",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        }
    },

    # DONE
    491273791: {
        True: {
            "россии",
            "настенн",
            "физическая",
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            "политич",
            'административн',
            'природные зоны',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        },
    },

    # DONE
    491273438: {
        True: {
            "росси",
            # "настенн",
            # "физическая",
            'природн.*зон',
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            "политич",
            'административн',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        },

    },

    # DONE
    491271768: {
        True: {
            "москв",
            # "путеводитель",
            "фальцован|складн",
        },
        False: {
            # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r"\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            'атлас автомобильных дорог',
            'английск',
            'черного моря',
            'две карты',
        },
        'include': [
            1688947923 # 'атлас автомобильных дорог'
        ]
    },

    # # DONE (no positives in dataset)
    # 491271339: {
    #     True: {
    #         "москв",
    #         # "путеводитель",
    #         "фальцован|складн",
    #         "англ",
    #     },
    #     False: {
    #         # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
    #         r"\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
    #         'атлас автомобильных дорог',
    #         'черного моря',
    #         'две карты',
    #     },
    # },

    # DONE
    491271320: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491271284: {
        True: {
            'физическ',
            'мир',
            'настенн',
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',
            
            'россии',
            
            'английск',
            'политическая',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'иллюстрированн|с иллюстрациями|с животными',
        }
    },

    # DONE
    491270369: {
        True: {
            "политическая|политико-административная|административно-политическая",
            "карта россии",
        },
        False: {
            # "тубус",
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491270272: {
        True: {
            "политическ",
            "мир",
            # "тубус",
            "ретро|старинн",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491268805: {
        True: {
            "москв",
            # "метро",
            'настенн',
        },
        False: {
            "фальцован|складн",
            'репринт'            
        }
    },
}

In [12]:
# CHOSEN_SKU = 1871769771
# CHOSEN_SKU = 1200553001
# CHOSEN_SKU = 491271768
# CHOSEN_SKU = 508611672
# CHOSEN_SKU = 508612558
# CHOSEN_SKU = 491271768
# CHOSEN_SKU = 491271339
# CHOSEN_SKU = 491271320
# CHOSEN_SKU = 491271284
# CHOSEN_SKU = 491270369
CHOSEN_SKU = 491270272
# CHOSEN_SKU = 491268805

chosen_subset_df = full_df

COLUMN_TO_MATCH = 'name_and_description'

MARGIN = 0.15

SHOW_ALL_ROWS = False
# SHOW_ALL_ROWS = True

In [20]:
# process_sku_matches

import operator
from functools import reduce
import pandas as pd

def process_sku_matches(
    chosen_sku: str,
    pattern_dict: dict,
    size_matches_df: pd.DataFrame,
    chosen_subset_df: pd.DataFrame,
    column_to_match: str,
    margin: float
):
    # 1) unpack the patterns for this SKU
    pos_patterns = pattern_dict.get(True,  [])
    neg_patterns = pattern_dict.get(False, [])

    # 2) build query_row with its true size
    query_size_df = size_matches_df[size_matches_df["sku"] == chosen_sku]
    q_w, q_h      = query_size_df.iloc[0][["width", "height"]]
    query_row     = chosen_subset_df[chosen_subset_df['sku'] == chosen_sku].copy()
    query_row['width'], query_row['height'] = q_w, q_h

    # 3) build a working regex‐flag DataFrame
    chosen_regex_df = chosen_subset_df.copy()
    for p in pos_patterns:
        col = f"P: {p}"
        chosen_regex_df[col] = chosen_regex_df[column_to_match]\
            .str.contains(p, case=False, na=False)
        query_row[col]       = query_row[column_to_match]\
            .str.contains(p, case=False, na=False)
    for n in neg_patterns:
        col = f"N: {n}"
        chosen_regex_df[col] = chosen_regex_df[column_to_match]\
            .str.contains(n, case=False, na=False)
        query_row[col]       = query_row[column_to_match]\
            .str.contains(n, case=False, na=False)

    # 4) pattern columns
    pattern_cols = [f"P: {p}" for p in pos_patterns] + [f"N: {n}" for n in neg_patterns]

    # 5) build masks
    pos_masks = [chosen_regex_df[c] for c in pattern_cols if c.startswith("P: ")]
    neg_masks = [~chosen_regex_df[c] for c in pattern_cols if c.startswith("N: ")]
    base_index = chosen_regex_df.index

    combined_mask = (
        reduce(operator.and_, pos_masks + neg_masks)
        if (pos_masks + neg_masks) else pd.Series(True, index=base_index)
    )
    pos_mask = (
        reduce(operator.and_, pos_masks)
        if pos_masks else pd.Series(True, index=base_index)
    )

    rejected  = chosen_regex_df.loc[~pos_mask   & (chosen_regex_df["sku"] != chosen_sku)].copy()
    positives = chosen_regex_df.loc[ combined_mask & (chosen_regex_df["sku"] != chosen_sku)]

    # 6) size‐matching helpers
    def best_match(col, sku):
        q_vals = size_matches_df.loc[size_matches_df["sku"] == chosen_sku, col].unique()
        c_vals = size_matches_df.loc[size_matches_df["sku"] == sku, col].unique()
        if not len(q_vals) or not len(c_vals):
            return None, None, float("inf")
        return min(
            ((c, q, abs(c - q) / q) for c in c_vals for q in q_vals),
            key=lambda x: x[2]
        )

    def add_size_cols(df, col):
        out = df.copy()
        triples = out["sku"].apply(lambda s: best_match(col, s))
        out[f"{col}_matched_cand"]  = triples.apply(lambda t: t[0])
        out[f"{col}_matched_query"] = triples.apply(lambda t: t[1])
        out[f"{col}_delta"]         = triples.apply(lambda t: t[2])
        return out

    def filter_by(col, df, inverse=False):
        tmp = add_size_cols(df, col)
        m   = tmp[f"{col}_delta"] <= margin
        return tmp[~m] if inverse else tmp[m]

    # 7) gather hard_negatives
    if neg_masks:
        neg_fail_mask = pos_mask & ~reduce(operator.and_, neg_masks)
        pattern_negs  = chosen_regex_df.loc[
            neg_fail_mask & (chosen_regex_df["sku"] != chosen_sku)
        ].copy()
    else:
        pattern_negs = pd.DataFrame(columns=chosen_regex_df.columns)

    neg_by_w = filter_by("width",  positives, inverse=True)
    neg_by_h = filter_by("height", positives, inverse=True)

    pattern_negs['fail_regex'] = True
    pattern_negs['fail_size']  = False

    neg_by_size = pd.concat([neg_by_w, neg_by_h])
    neg_by_size['fail_size']   = True

    hard_negatives = (
        pd.concat([pattern_negs, neg_by_size])
        .drop_duplicates(subset="sku")
        .pipe(add_size_cols, col="width")
        .pipe(add_size_cols, col="height")
    )
    hard_negatives['fail_regex'] = hard_negatives['fail_regex'].astype('boolean').fillna(False)

    # 8) re‐filter positives by size
    positives = filter_by("width",  positives)
    positives = filter_by("height", positives)

    # 9) assemble display_cols
    size_cols = [
        "width_matched_cand","width_matched_query","width_delta",
        "height_matched_cand","height_matched_query","height_delta"
    ]
    fail_cols = ["fail_regex","fail_size"]

    return positives, hard_negatives, rejected, query_row, fail_cols, size_cols, pattern_cols

In [15]:
# print_sku_matches

def print_sku_matches(
    positives: pd.DataFrame,
    hard_negatives: pd.DataFrame,
    rejected: pd.DataFrame,
    query_row: pd.DataFrame,
    display_cols: list,
    chosen_sku: str,
    chosen_sku_idx: int = None,
    show_all_rows: bool = False
):
    def show(df, cols):
        cols = [c for c in cols if c in df.columns]
        display(df[cols])

    if show_all_rows:
        pd.set_option("display.max_rows", None)
    else:
        pd.reset_option("display.max_rows")
    pd.set_option("display.max_colwidth", None)

    if chosen_sku_idx is not None:
        print(f"\n📌 Query SKU {chosen_sku} (#{chosen_sku_idx}):")
    else:
        print(f"\n📌 Query SKU {chosen_sku}:")

    show(query_row, ['sku','name','width','height'] + display_cols)

    print(f"\n✅ Positives (count: {len(positives)}):")
    show(positives, ['sku','name'] + display_cols)

    print(f"\n❌ Hard negatives (count: {len(hard_negatives)}):")
    show(hard_negatives, ['sku','name'] + display_cols)

    print(f"\n🚫 Simple negative (rejected) (count: {len(rejected)}):")
    show(rejected, ['sku','name'] + display_cols)

In [23]:
# Process sku matches

(
    positives,
    hard_negatives,
    rejected,
    query_row,
    fail_cols,
    size_cols,
    pattern_cols
) = process_sku_matches(
    chosen_sku       = CHOSEN_SKU,
    pattern_dict     = ALL_PATTERNS_DICT[CHOSEN_SKU],
    size_matches_df  = size_matches_df,
    chosen_subset_df = chosen_subset_df,
    column_to_match  = COLUMN_TO_MATCH,
    margin           = MARGIN
)


In [24]:
# Print matches

print_sku_matches(
    positives, hard_negatives, rejected,
    query_row,

    display_cols=fail_cols + size_cols,
    # display_cols=fail_cols + pattern_cols,

    chosen_sku=CHOSEN_SKU,
    show_all_rows = SHOW_ALL_ROWS,
)


📌 Query SKU 491270272:


,sku,name,width,height
5279,491270272,"Политическая карта мира настенная ""Мир в стиле ретро"" 160х109см, Постер ""Карта мира"", Выпуск 2024 год",160,109



✅ Positives (count: 14):


,sku,name,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta
327,1849926941,Атлас Принт Настенная политическая карта мира в стиле ретро 1:22 млн,154,160,0.03750,107,109,0.018349
647,1776962648,"Политич. карта мира (ретро-стиль), ламинир.",138,160,0.13750,96,109,0.119266
783,1757684675,Атлас Принт Настенная политическая карта мира в стиле ретро 1:22 млн,154,160,0.03750,107,109,0.018349
2786,1406174525,Настенная карта Мир политическая 1:22млн/размер 157х107 см/на отвесах/ новые границы,157,160,0.01875,107,109,0.018349
3529,1011784724,"Настенная карта Мир политическая 1:22млн.,1,54х1,07м.,ретро стиль - 1 ШТ.",154,160,0.03750,107,109,0.018349
3544,1005611591,"Карта Мира политическая в стиле ретро 120 х 180 см, GlobusOff",180,160,0.12500,120,109,0.100917
3718,945075481,"Атлас Принт Географическая карта 100 x 149 см, масштаб: 1:22 000 000",154,160,0.03750,107,109,0.018349
3731,943273414,"Карта Мира политическая в стиле ретро 160 х 110 см, GlobusOff",160,160,0.00000,110,109,0.009174
3883,855023860,Настенная карта Мир политическая 1:22млн/размер 160х109 см/ретро стиль/ новые границы,160,160,0.00000,109,109,0.000000
4131,733221245,Атлас Принт Географическая карта,154,160,0.03750,107,109,0.018349



❌ Hard negatives (count: 24):


,sku,name,fail_regex,fail_size,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta
1610,1677371389,"Карта мира на английском языке в ретро стиле с новыми границами России в тубусе, настенная, 120х80 см, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,120,160,0.25000,80,109,0.266055
1775,1642067085,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,109,0.266055
2301,1540069061,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.25000,80,109,0.266055
2302,1540065800,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,109,0.266055
2309,1538722406,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.25000,80,109,0.266055
2311,1536520050,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,109,0.266055
3821,891300230,"Складная карта ""Мир и Россия. Карты географических открытий""",True,False,100,160,0.37500,70,109,0.357798
4785,601625566,"Административная карта АГТ Геоцентр, масштаб: 1:25 000 000",True,False,160,160,0.00000,120,109,0.100917
4786,601625531,"Карта мира большая в ретро стиле с новыми границами России 160х120 см в тубусе, матовая ламинация, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,160,160,0.00000,120,109,0.100917
4791,601560335,"Карта мира на английском языке в ретро стиле на отвесах (на рейках) в тубусе, настенная, 120х80 см, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,120,160,0.25000,80,109,0.266055



🚫 Simple negative (rejected) (count: 5523):


,sku,name
0,1871769771,"Карты МИРА и РОССИИ настенные политические,160х102 см, Комплект школьных карт для детей / 2024 г"
1,1679550303,"Схема линий скоростного транспорта Москвы (Метро, МЦК, МЦД) 0,5*0,7 м, ламинированная"
2,1200553001,"Политическая карта МИРА 160х109 см, Карта мира настенная, подарок для школьника"
3,922231521,"Политическая карта МИРА настенная, 100х70см, школьная географическая карта мира, Выпуск 2024 год, В ТУБУСЕ"
4,922230517,"Политическая карта МИРА настенная, 160х102см, карта мира по географии, Выпуск 2024 год, В ТУБУСЕ"
...,...,...
5557,166584090,"Карта настенная Мир. Политическая карта с флагами, М-1:30 млн., размер 122х79 см, ламинированная"
5558,166451882,"Карта мира, магнитно-маркерная, 60х40 см, комплект цветных маркеров со стирателем"
5559,154409524,Карта звездного неба космоса настенная светящаяся для детей в подарок Gagarin Map
5560,147896031,Скретч карта мира географическая Dark Edition (80х60см) в тубусе


In [ ]:
# View particular sku in a particular subset dataframe

SKU_TO_VIEW = 1677371389

pd.set_option('display.max_columns', None)

full_df[full_df.sku == SKU_TO_VIEW][['sku', 'name', 'description']]

cols_to_drop = ['sku', 'name'] + fail_cols + size_cols + pattern_cols

# size_matches_df[size_matches_df.sku == SKU_TO_VIEW]

hard_negatives[
#     hard_negatives.sku == SKU_TO_VIEW

    # (hard_negatives.fail_regex)
    (~hard_negatives.fail_size) & (hard_negatives.fail_regex)
    # hard_negatives['N: окрестн']
    ][
        cols_to_drop    
    ]

# positives[positives.sku == SKU_TO_VIEW][
#         ['sku','name',
#         # 'width_matched_cand', 'height_matched_cand', 'width_delta','height_delta',
#         ] + display_cols
# ]

# rejected[rejected.sku == ][['sku', 'name']]

,sku,name,fail_regex,fail_size,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta,P: ретро|старинн,P: мир,P: политическ,N: полушар,"N: комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",N: фотообои,N: карта россии,N: фальцован|складн,N: административная,N: английск,N: скретч,N: физическ
1610,1677371389,"Карта мира на английском языке в ретро стиле с новыми границами России в тубусе, настенная, 120х80 см, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,False,False,False,False,False,True,False,False
1775,1642067085,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.2500,80,109,0.266055,True,True,True,False,True,False,True,False,False,False,False,False
2301,1540069061,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,True,False,False,False,True,False,False,False
2302,1540065800,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,True,False,False,False,True,False,False,False
2309,1538722406,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,True,False,False,False,True,False,False,False
2311,1536520050,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,True,False,False,False,True,False,False,False
3821,891300230,"Складная карта ""Мир и Россия. Карты географических открытий""",True,False,100,160,0.3750,70,109,0.357798,True,True,True,False,False,False,False,True,False,False,False,False
4785,601625566,"Административная карта АГТ Геоцентр, масштаб: 1:25 000 000",True,False,160,160,0.0000,120,109,0.100917,True,True,True,True,False,False,True,False,True,False,False,False
4786,601625531,"Карта мира большая в ретро стиле с новыми границами России 160х120 см в тубусе, матовая ламинация, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,160,160,0.0000,120,109,0.100917,True,True,True,True,False,False,True,False,False,False,False,False
4791,601560335,"Карта мира на английском языке в ретро стиле на отвесах (на рейках) в тубусе, настенная, 120х80 см, для дома, офиса, школы, ""АГТ Геоцентр""",True,False,120,160,0.2500,80,109,0.266055,True,True,True,True,False,False,False,False,False,True,False,False


In [29]:
# # Check price of query vs positives
# %matplotlib inline
# import matplotlib.pyplot as plt

# # Display query SKU information
# display(query_row[['sku', 'final_price']])

# # Plot histogram of positive SKUs' prices
# ax = positives['final_price'].plot(kind='hist', bins=30, alpha=0.7, title='Price Distribution of Positive Matches')
# # Add a vertical line for the query SKU's price
# ax.axvline(query_row['final_price'].values[0], color='red', linestyle='dashed', linewidth=2, label='Query Price')

# # Add legend and labels
# ax.set_xlabel('Final Price')
# ax.set_ylabel('Count')
# ax.legend()
# plt.show()


### Pairwise dataset

In [54]:
# make_pairwise_for_query

import pandas as pd

def make_pairwise_for_query(
    query_row: pd.DataFrame,
    positives: pd.DataFrame,
    hard_negatives: pd.DataFrame,
    cols_to_drop: list,
) -> pd.DataFrame:
    """
    Builds a pairwise DataFrame for one query using vectorized pandas ops:
      - query columns get suffix '_query'
      - candidate columns get suffix '_candidate'
      - only a 'label' column beyond those
    Drops any columns listed in display_cols from both sides.
    """
    # 1) prepare query side
    q = (
        query_row
        .drop(columns=cols_to_drop, errors='ignore')
        .iloc[[0]]                             # keep as single-row DF
        .rename(columns=lambda c: f"{c}_query")
        .assign(_tmpkey=1)
    )

    # 2) prepare positives
    pos = (
        positives
        .drop(columns=cols_to_drop, errors='ignore')
        .rename(columns=lambda c: f"{c}_candidate")
        .assign(_tmpkey=1)
    )
    pos_pairs = (
        pd.merge(q, pos, on="_tmpkey")
        .drop(columns=["_tmpkey"])
        .assign(label=1)
    )

    # 3) prepare hard_negatives
    neg = (
        hard_negatives
        .drop(columns=cols_to_drop, errors='ignore')
        .rename(columns=lambda c: f"{c}_candidate")
        .assign(_tmpkey=1)
    )
    neg_pairs = (
        pd.merge(q, neg, on="_tmpkey")
        .drop(columns=["_tmpkey"])
        .assign(label=0)
    )

    # 4) concatenate
    return pd.concat([pos_pairs, neg_pairs], ignore_index=True)


In [55]:
# — build the full pairwise dataset —
from tqdm import tqdm

all_pairs = []

for query_sku, pattern_dict in tqdm(ALL_PATTERNS_DICT.items()):
    (
        positives,
        hard_negatives,
        rejected,
        query_row,
        fail_cols,
        size_cols,
        pattern_cols
    ) = process_sku_matches(
        chosen_sku       = CHOSEN_SKU,
        pattern_dict     = ALL_PATTERNS_DICT[CHOSEN_SKU],
        size_matches_df  = size_matches_df,
        chosen_subset_df = chosen_subset_df,
        column_to_match  = COLUMN_TO_MATCH,
        margin           = MARGIN
    )

    df_pairs = make_pairwise_for_query(
        query_row, positives, hard_negatives,
        cols_to_drop=fail_cols + size_cols + pattern_cols
    )
    all_pairs.append(df_pairs)
    break

pairwise_dataset = pd.concat(all_pairs, ignore_index=True)

  0%|          | 0/20 [00:01<?, ?it/s]


In [63]:
print("Total entries:", pairwise_dataset.shape[0])
print("Total query SKU:", pairwise_dataset.sku_query.nunique())
print("Columns:")
pairwise_dataset.columns.tolist()

Total entries: 38
Total query SKU: 1
Columns:


['sku_query',
 'description_query',
 'image_url_query',
 'name_query',
 'category_query',
 'схема_query',
 'brand_query',
 'niche_query',
 'seller_query',
 'balance_fbo_query',
 'balance_fbs_query',
 'warehouses_count_query',
 'comments_query',
 'final_price_query',
 'max_price_query',
 'min_price_query',
 'average_price_query',
 'median_price_query',
 'membership_card_price_query',
 'sales_query',
 'revenue_query',
 'revenue_potential_query',
 'revenue_average_query',
 'lost_profit_query',
 'lost_profit_percent_query',
 'url_query',
 'thumb_query',
 'pics_count_query',
 'has_video_query',
 'first_date_query',
 'days_in_website_query',
 'days_in_stock_query',
 'days_with_sales_query',
 'average_if_in_stock_query',
 'rating_query',
 'fbs_query',
 'base_price_query',
 'category_position_query',
 'categories_last_count_query',
 'sales_per_day_average_query',
 'frozen_stocks_query',
 'frozen_stocks_cost_query',
 'frozen_stocks_percent_query',
 'balance_query',
 'image_name_query',
 'name_a

In [64]:
from joblib import hash

pattern_dict_hash = hash(ALL_PATTERNS_DICT)
file_name = (
    'regex-pairwise-dataset'
    f'_num-queries={pairwise_dataset.sku_query.nunique()}'
    f'_num-pairs={len(pairwise_dataset)}'
    f'_patterns-dict-hash={pattern_dict_hash}'
    '.csv'
)
print(file_name)

file_path = (
    Path('../clip-siamese') / 'data' / 'tables_OZ_geo_5500' /
    'processed' / file_name
)

file_path.parent.mkdir(parents=True, exist_ok=True)
pairwise_dataset.to_csv(file_path, index=False)

regex-pairwise-dataset_num-queries=1_num-pairs=38_patterns-dict-hash=6dbf9b3ef9568e60cd959f87be7e3b26.csv
